In [1]:
from scipy import spatial, sparse
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import textacy

ImportError: /home/immersinn/anaconda3/lib/python3.5/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.21' not found (required by /home/immersinn/anaconda3/lib/python3.5/site-packages/spacy/tokenizer.cpython-35m-x86_64-linux-gnu.so)

In [ ]:
import sys

In [ ]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [ ]:
import mysql_utils
import events_merge

In [ ]:
limit = 15

In [ ]:
cnx = mysql_utils.getCnx()
cur = mysql_utils.getCur(cnx)

In [ ]:
query_text = '''SELECT * FROM {} LIMIT {}'''.format(mysql_utils.TABLE, limit)

In [ ]:
cur.execute(query_text)

In [ ]:
btf = lambda doc: events_merge.build_text_feature(doc, 
                                                  components=['title', 'summary'],
                                                  lower=False, remove_stops=False,
                                                  html_text=True)

In [ ]:
def doc2Dict(raw_doc, cursor):
    return({k : f for f,k in zip(raw_doc, cursor.column_names)})

In [ ]:
doc_dets = [doc2Dict(doc, cur) for doc in cur]

In [ ]:
doc_dets

In [ ]:
feature = [btf(doc) for doc in doc_dets]

In [ ]:
count_vect = CountVectorizer()

In [ ]:
X = count_vect.fit_transform([feature[0]])

In [ ]:
X.todense()

In [ ]:
count_vect.vocabulary_